In [21]:
import numpy as np
import math
import scipy
from scipy import special
import random
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
from scipy.optimize import fmin_slsqp
import scipy.io
import time

# Parameters & Data

In [22]:
e = math.e

optimal_tolerance = 0.00001

l = 160
beta = 3200
J_max = 10
schedule_frameTime = 5
max_epoch = 10


### RBs & Beams
N_sub6 = 2
N_mmW = 2

bandwidth_s6 = 1e+8; #Bandwidth sub6 =  1e8 Hz = 100 MHz
bandwidth_mmW = 1e+9; #Bandwidth mmWave = 1e9 Hz = 1 GHz
band_per_RB = bandwidth_s6 / N_sub6; 

In [23]:
## QoS requirements
users_infos_filename = 'MATLAB/ressources/users_infos_K4.mat'

## Channels & LoS
los_filename = 'MATLAB/ressources/topologies/topo_3x3/4users/nLoS.mat'

## Predefined Channels
channels_filename = 'MATLAB/ressources/topologies/topo_3x3/4users/Same_Channels_K4_nLoS.mat'


users_infos_file = scipy.io.loadmat(users_infos_filename)
los_file = scipy.io.loadmat(los_filename)
channels_file = scipy.io.loadmat(channels_filename)

## QoS Requirements
users_infos = users_infos_file['users_infos']

## Channels
SINR_S6_allframes = los_file['SINR_sub6_central_users']
SINR_mmW_allframes = los_file['SINR_mmW_central_users']
los = los_file['los_central_users']

users_infos[0,:] = los

## Predefined Channels
SINR_s6_SameChan = channels_file['SINR_sub6_value']
SINR_mmW_SameChan = channels_file['SINR_mmW_value']
#LoS_SameChan = np.array(channels_file['LoS_mmW'])


'''
users_infos[0]: LoS Proba. 
users_infos[1]: Required rate 
users_infos[2]: Achieved Rate (initially 0) 
users_infos[3]: Required Delay
users_infos[4]: Achieved Delay (Initially 0)
users_infos[5]: Taux d'erreur accepté (1e-5)
'''
print("Users Infos:")
print(users_infos, "\n")

Users Infos:
[[1.00000000e+00 8.56963734e-01 9.72183169e-01 8.11777547e-01]
 [1.00000000e+08 5.00000000e+07 5.00000000e+08 1.00000000e+07]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.00000000e+00 5.00000000e+00 3.00000000e+00 4.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.00000000e-05 1.00000000e-05 1.00000000e-05 1.00000000e-05]] 



# Fonctions utilisées

### Utilitaires

In [24]:
'''
Create sub groups
    @in : partition
    @out : 3 binary matrix, one for each frequency
'''
def create_subgroups(partition):
    sub6 = np.zeros(len(partition))
    mmW = np.zeros(len(partition))
    unpartitioned = np.zeros(len(partition))
    
    for i in range (len(partition)):
        if(partition[i] == 1):
            sub6[i] = 1
        elif(partition[i] == 2):
            mmW[i] = 1
        else:
            unpartitioned[i] = 1
            
    return sub6.astype(int), mmW.astype(int), unpartitioned.astype(int)

#Sample SINR matrix for max_epoch frames
def datasampler(SINR, max_epoch, K): 
    max_size = len(SINR[0])
    mask = np.random.randint(0, max_size, max_epoch)
    
    res = []
    for i in range(int(K)):
        res.append(SINR[i][mask])
        
    return res
                    
#user infos (los, rate required, error req, delay req) 
def users_info_freq(partition, K):
    s6 = []
    mmW = []
    for i in range(K):
        los = users_infos[0][i]
        rate_req = users_infos[1][i]
        error_req = users_infos[5][i]
        delay_req = users_infos[3][i]
        if(partition[i] == 1):    
            s6.append([los, rate_req, error_req, delay_req])
        if(partition[i] == 2):
            mmW.append([los, rate_req, error_req, delay_req])
    
    return s6, mmW

#To calcul the average delay 
def delay(schedule, K, N, schedule_frameTime):
    '''
    Returns the list of the last used slot for each user in the band.
    '''
    schedule = np.array(transform_array(schedule, K, N))
    delay = np.zeros(K)
    # print("schedule",schedule)
    for k in range(K):
        for x in range(schedule_frameTime):
            # print(x,k,schedule[:, x, k])
            if np.sum(schedule[:, x, k]) >= 1:
                delay[k] = x
    return delay

## Functions from the paper

### Rate

In [25]:
## Channel Capacity

### A multiplier par la bande disponible et ratio slot utilisé (cf. fonction F dans code matlab)
def F(schedule, SINR, band, scheduling_frameTime, N, K):
    ratio_slot_used = ratio_slot(transform_array(schedule,K,N), scheduling_frameTime, N, K)
    size = len(schedule)
    nbBlock = int(size/K)
    capacity = np.zeros(size)
    for i in range (K):
        capacity[i*nbBlock:(i+1)*nbBlock] = schedule[i*nbBlock:(i+1)*nbBlock] * np.log(1+SINR)[i]
    return np.sum(capacity) * band * ratio_slot_used

def F_obj(schedule, SINR, K):
    size = len(schedule)
    nbBlock = int(size/K)
    capacity = np.zeros(size)
    for i in range (K):
        capacity[i*nbBlock:(i+1)*nbBlock] = schedule[i*nbBlock:(i+1)*nbBlock] * np.log(1+SINR)[i]
    return np.sum(capacity)

def ratio_slot(sched, scheduling_frameTime, N, K):
    '''
    Returns the list of the average ratio of used slots in all N for each user.
    '''
    ratio_slot_used_k = np.zeros((K, N))
    slot_used_k = np.zeros(K)

    for k in range(K):
        for n in range(N):
            ratio_slot_used_k[k, n] = np.sum(sched[n, :, k]) / scheduling_frameTime

        slot_used_k[k] = np.sum(ratio_slot_used_k[k, :]) / N

    return slot_used_k

#Helper function to compute the Capacity (F value) of each user and stored in a size
#K_sub6 array
### A multiplier par la bande disponible et ratio slot utilisé (cf. fonction F dans code matlab)
def F_k(schedule, SINR, band, scheduling_frameTime, N, K):
    ratio_slot_used = ratio_slot(transform_array(schedule,K,N), scheduling_frameTime, N, K)
    size = len(schedule)
    nbBlock = int(size/K)
    capacity = np.zeros(size)
    res = []
    for i in range (K):
        capacity[i*nbBlock:(i+1)*nbBlock] = schedule[i*nbBlock:(i+1)*nbBlock] * np.log(1+SINR)[i]
        res.append(np.sum(capacity[i*nbBlock:(i+1)*nbBlock]))
    return np.array(res) * band * ratio_slot_used



## Channel dispersion

### A multiplier par la bande disponible et ratio slot utilisé (cf. fonction V dans code matlab)
def V(schedule, SINR, Q_inv, l, band, scheduling_frameTime, N, K):
    ratio_slot_used = ratio_slot(transform_array(schedule,K,N), scheduling_frameTime, N, K)
    size = len(schedule)
    nbBlock = int(size/K)
    
    dispersion = np.ones(size)
    square = np.zeros(K)
    for i in range(K):
        dispersion[i*nbBlock:(i+1)*nbBlock] = schedule[i*nbBlock:(i+1)*nbBlock] * np.log(e)**2 * (1 - 1/((1 + SINR)**2))[i]
        square[i] = np.sqrt(np.sum(dispersion[i*nbBlock:(i+1)*nbBlock]))
        
    dispersion_user_k = (Q_inv/np.sqrt(l)) * square
    
    return np.sum(dispersion_user_k) * band * ratio_slot_used

def V_obj(schedule, SINR, Q_inv, l, K):
    size = len(schedule)
    nbBlock = int(size/K)
    
    dispersion = np.ones(size)
    square = np.zeros(K)
    for i in range(K):
        dispersion[i*nbBlock:(i+1)*nbBlock] = schedule[i*nbBlock:(i+1)*nbBlock] * np.log(e)**2 * (1 - 1/((1 + SINR)**2))[i]
        square[i] = np.sqrt(np.sum(dispersion[i*nbBlock:(i+1)*nbBlock]))
        
    dispersion_user_k = (Q_inv/np.sqrt(l)) * square 
    
    return np.sum(dispersion_user_k)

#Helper function to compute the Dispersion (V value) of each user and stored in a size
#K_sub6 array
### A multiplier par la bande disponible et ratio slot utilisé (cf. fonction V_k dans code matlab)
def V_k(schedule, SINR, Q_inv, l, band, scheduling_frameTime, N, K):
    size = len(schedule)
    nbBlock = int(size/K)
    ratio_slot_used = ratio_slot(transform_array(schedule,K,N), scheduling_frameTime, N, K)
    
    dispersion = np.ones(size)
    square = np.zeros(K)
    for i in range(K):
        dispersion[i*nbBlock:(i+1)*nbBlock] = schedule[i*nbBlock:(i+1)*nbBlock] * np.log(e)**2 * (1 - 1/((1 + SINR)**2))[i]
        square[i] = np.sqrt(np.sum(dispersion[i*nbBlock:(i+1)*nbBlock]))
        
    dispersion_user_k = (Q_inv/np.sqrt(l)) * square 
    
    return dispersion_user_k * band * ratio_slot_used

In [26]:
def W(schedule):
    return np.sum(schedule, axis = (0))

def E(schedule):
    schedule_squared = schedule * schedule
    return np.sum(schedule_squared, axis = (0))

In [27]:
def gradE(schedule, schedule_init):
    
    double_sched = 2*schedule
    difference = schedule - schedule_init
    whole = double_sched*difference
    return np.sum(whole, axis = (0))

def inv_ifNotZero(arr):
    return [1/x if x != 0 else 0 for x in arr]


def gradV_obj(schedule, schedule_init, SINR, Q_inv, l, K):
    size = len(schedule)
    nbBlock = int(size/K)
    
    dispersion = np.ones(size)
    square = np.zeros(K)
    for i in range(K):
        dispersion[i*nbBlock:(i+1)*nbBlock] = schedule_init[i*nbBlock:(i+1)*nbBlock] * np.log(e)**2 * (1 - 1/((1 + SINR)**2))[i]
        square[i] = np.sqrt(np.sum(dispersion[i*nbBlock:(i+1)*nbBlock]))
    
    gradient_leftPart = (Q_inv/np.sqrt(l)) * inv_ifNotZero(square)
    
    gradient_rightPart = np.array( [(1/2) * np.log(e)**2 * (1 - 1/((1+ x)**2)) for x in SINR])
    
    gradMatrix_oneFrame = np.array( [gradient_leftPart[idx]*perUser_rightPart for idx,perUser_rightPart in enumerate(gradient_rightPart)])
    
    difference = schedule - schedule_init
    
    full_gradMatrix = np.zeros(size)
    for i in range(K):
        full_gradMatrix[i*nbBlock:(i+1)*nbBlock] = difference[i*nbBlock:(i+1)*nbBlock] * gradMatrix_oneFrame[i]
    
    return np.sum(full_gradMatrix)

### A multiplier par la bande disponible et ratio slot utilisé (cf. fonction gradV dans code matlab)
def gradV(schedule, schedule_init, SINR, Q_inv, l, band, schedule_frameTime, N, K):
    ratio_slot_used = ratio_slot(transform_array(schedule,K,N), schedule_frameTime, N, K)
    size = len(schedule)
    nbBlock = int(size/K)
    
    dispersion = np.ones(size)
    square = np.zeros(K)
    for i in range(K):
        dispersion[i*nbBlock:(i+1)*nbBlock] = schedule_init[i*nbBlock:(i+1)*nbBlock] * np.log(e)**2 * (1 - 1/((1 + SINR)**2))[i]
        square[i] = np.sqrt(np.sum(dispersion[i*nbBlock:(i+1)*nbBlock]))
    
    gradient_leftPart = (Q_inv/np.sqrt(l)) * inv_ifNotZero(square)
    
    gradient_rightPart = np.array( [(1/2) * np.log(e)**2 * (1 - 1/((1+ x)**2)) for x in SINR])
    
    gradMatrix_oneFrame = np.array( [gradient_leftPart[idx]*perUser_rightPart for idx,perUser_rightPart in enumerate(gradient_rightPart)])
    
    difference = schedule - schedule_init
    
    full_gradMatrix = np.zeros(size)
    for i in range(K):
        full_gradMatrix[i*nbBlock:(i+1)*nbBlock] = difference[i*nbBlock:(i+1)*nbBlock] * gradMatrix_oneFrame[i]
    
    return np.sum(full_gradMatrix) * band * ratio_slot_used

#Helper function to compute the Dispersion (V value) of each user and stored in a size
#K_sub6 array
### A multiplier par la bande disponible et ratio slot utilisé (cf. fonction gradV_k dans code matlab)
def gradV_k(schedule, schedule_init, SINR, Q_inv, l, band, schedule_frameTime, N, K):
    ratio_slot_used = ratio_slot(transform_array(schedule,K,N), schedule_frameTime, N, K)
    size = len(schedule)
    nbBlock = int(size/K)
    
    dispersion = np.ones(size)
    square = np.zeros(K)
    for i in range(K):
        dispersion[i*nbBlock:(i+1)*nbBlock] = schedule_init[i*nbBlock:(i+1)*nbBlock] * np.log(e)**2 * (1 - 1/((1 + SINR)**2))[i]
        square[i] = np.sqrt(np.sum(dispersion[i*nbBlock:(i+1)*nbBlock]))
    
    gradient_leftPart = (Q_inv/np.sqrt(l)) * inv_ifNotZero(square)
    
    gradient_rightPart = np.array( [(1/2) * np.log(e)**2 * (1 - 1/((1+ x)**2)) for x in SINR])
    
    gradMatrix_oneFrame = np.array( [gradient_leftPart[idx]*perUser_rightPart for idx,perUser_rightPart in enumerate(gradient_rightPart)])
    
    difference = schedule - schedule_init
    
    full_gradMatrix = np.zeros(size)
    res = []
    for i in range(K):
        full_gradMatrix[i*nbBlock:(i+1)*nbBlock] = difference[i*nbBlock:(i+1)*nbBlock] * gradMatrix_oneFrame[i]
        res.append(np.sum(full_gradMatrix[i*nbBlock:(i+1)*nbBlock]))

    return np.array(res) * band * ratio_slot_used

## Objective & Constraints

In [28]:
### Objective
def U_bar(schedule, schedule_init, SINR, Q_inv, l, K):
    return -F_obj(schedule, SINR, K) + V_obj(schedule_init, SINR, Q_inv, l, K) + gradV_obj(schedule, schedule_init, SINR, Q_inv, l, K) + beta*(W(schedule) - E(schedule_init) - gradE(schedule, schedule_init)) 

In [29]:
## CONSTRAINTS : (A vérifier)
def C_delay(schedule, K, user_infos):
    cpt = 0
    res = []
    size = len(schedule)
    nbBlock = int(size / K)
    for k in range(K):
        for i in range(cpt, cpt+nbBlock, 5):
        
            for idx, x in enumerate(schedule[i:i+schedule_frameTime]):
                if(idx + 1 > user_infos[k,3]):
                    res.append(int(schedule[idx+i]))
        cpt += nbBlock
    return -np.sum(res)

def C_delay2(schedule, K, user_infos):
    ceq = []
    for k in range(K):
        delay_k = user_infos[k,3]
        cons_delay_k = schedule[:, delay_k + 1:, k]
        ceq += cons_delay_k.tolist()
    return ceq

def C_delay(schedule, K, user_infos, N):
    schedule_frameTime = int(len(schedule)/(K*N))
    res = 0
    for k in range(K):
        for n in range(N):
            for t in range(int(user_infos[k,3]), schedule_frameTime):
                res += schedule[n*schedule_frameTime*K + t*K + k]
    return res

# def C_delay(schedule, K, users_infos):
#     constraints = []

#     for k in range(K):
#         delay_k = users_infos[3, k]  # Assuming delay information is in the 4th row of users_infos
#         cons_delay_k = schedule[delay_k:, k]  # Selecting rows from delay_k to the end for user k
#         constraints.extend(cons_delay_k)

#     return constraints

def C_rate(schedule, schedule_init, SINR, Q_inv, l, K, user_infos, band_per_RB, schedule_frameTime, N):
    return F_k(schedule, SINR, band_per_RB, schedule_frameTime, N, K) - V_k(schedule_init, SINR, Q_inv, l, band_per_RB, schedule_frameTime, N, K) - gradV_k(schedule, schedule_init, SINR, Q_inv, l, band_per_RB, schedule_frameTime, N, K) - user_infos[:,1]
    
def C_relaxation(schedule):
    return W(schedule) - E(schedule)
    
def C_blockPerUser(schedule, N, K):
    nb_case = schedule_frameTime * N
    res = []
    temp = 0
    for i in range (nb_case):
        for k in range(K):
            temp += schedule[i + nb_case*k]
        res.append(1 - temp)         
        temp = 0
    return np.array(res) 
    
    
def C_blockPerFrame(schedule, N):
    res = []
    size = len(schedule)
    for i in range (schedule_frameTime):
        res.append(N - np.sum(schedule[i:size:schedule_frameTime]))
    return np.array(res)

def C_zero(schedule):
    return schedule

def C_one(schedule):
    return 1-schedule 

def transform_array(schedule, K, N):
    array = np.zeros((N,schedule_frameTime,K))
    cpt = 0
    for n in range(N):
        for t in range(schedule_frameTime):
            for k in range(K):
                array[n][t][k] = schedule[cpt]
                cpt += 1
    return array


def AllCons(schedule_init, SINR, Q_inv, l, K, N, users_infos, band_per_RB):
    cons = []
    #Bounds : 
    cons += [{'type' : 'ineq', 'fun': C_zero}, {'type' : 'ineq', 'fun': C_one}]

    c_1 = {'type' : 'ineq', 'fun': C_relaxation}
    cons += [c_1]

    #Bloc User
    c_2 = {'type' : 'ineq', 'fun': C_blockPerUser, 'args':(N, K)}
    cons += [c_2]  

    #Block Frame
    c_3 = {'type' : 'ineq', 'fun' : C_blockPerFrame, 'args':([N])}
    cons += [c_3]

    #Delay
    # c_delay = {'type':'ineq', 'fun' : C_delay, 'args':(K, users_infos)}
    c_delay = {'type':'eq', 'fun' : C_delay, 'args':(K, users_infos, N)}
    cons += [c_delay]

    #Rate
    c_rate = {'type':'ineq', 'fun': C_rate, 'args': (schedule_init, SINR, Q_inv, l, K, users_infos, band_per_RB, schedule_frameTime, N)}
    cons += [c_rate] 
    
    return cons

def init_schedule3(N, K, schedule_frameTime):
    # sched_init = np.zeros((N, schedule_frameTime, K), dtype=int)
    sched_init = np.zeros(N * schedule_frameTime * K)
    if K == 0:
        return sched_init
    cpt_k = 0
    t = 0
    cpt_n = 0
    for i in range(N * schedule_frameTime):
        sched_init[cpt_n*schedule_frameTime*K + t*K + cpt_k] = 1
        # sched_init[cpt_n,t,cpt_k] = 1
        if (t + 1) % schedule_frameTime == 0:
            t = 0
            cpt_n += 1
        else:
            t += 1
        if (cpt_k + 1) % K == 0:
            cpt_k = 0
        else:
            cpt_k += 1
    return sched_init

def init_schedule2(N, K, schedule_frameTime):
    # sched_init = np.zeros((N, schedule_frameTime, K), dtype=int)
    sched_init = np.zeros(N * schedule_frameTime * K)
    if K == 0:
        return sched_init
    for t in range(schedule_frameTime):
        for cpt_n in range(N):
            k_possible = []
            for cpt_k in range(K):
                if t<users_infos[3,cpt_k]:
                    k_possible.append(cpt_k)
            if k_possible:
                ind = random.randint(0,len(k_possible)-1)
                sched_init[cpt_n*schedule_frameTime*K + t*K + k_possible[ind]] = 1
    # print(N,schedule_frameTime,K)
    return sched_init


# Scheduling

In [30]:
def scheduler(partition, iter_epoch, iter_learning):
    
    start_time = time.time()
    
    #Create sub groups :
    sub6_users, mmW_users, unpartitioned_users = create_subgroups(partition)
    
    K_total = len(partition)
    K_sub6 = sum(sub6_users)
    K_mmW = sum(mmW_users)
    
    ## Récupérer les valeurs des canaux pour l'itération iter_learning de l'episode iter_epoch
    ## 2D matrix, chaque ligne correspond aux valeurs des canaux pour une trame pour chaque user
    SINR_sub6_value = SINR_s6_SameChan[:,:,iter_learning, iter_epoch]
    SINR_mmW_value = SINR_mmW_SameChan[:,:,iter_learning, iter_epoch]
    
    
    # Séparer les SINR des mmW et Sub-6 users pour les 10 prochaines trames
    SINR_sub6 = []
    SINR_mmW = []
    for i in range (K_total):
        if(partition[i] == 1):
            # SINR_sub6.append(SINR_S6_allframes[:,i])
            SINR_sub6.append(SINR_sub6_value[:,i])
        if(partition[i] == 2):
            # SINR_mmW.append(SINR_mmW_allframes[:,i])
            SINR_mmW.append(SINR_mmW_value[:,i])
    
        
    SINR_sub6 = np.array(SINR_sub6)
    SINR_mmW = np.array(SINR_mmW)
        

    #Create all the informations about the subgroup necessary for the scheduler
    users_infos_sub6, users_infos_mmW = users_info_freq(partition, K_total)
    Q_inv_sub6 = np.array([np.sqrt(2) * special.erfinv(1 - 2*x[2]) for x in users_infos_sub6])
    Q_inv_mmW = np.array([np.sqrt(2) * special.erfinv(1 - 2*x[2]) for x in users_infos_mmW])

    users_infos_sub6 = np.array(users_infos_sub6)
    users_infos_mmW = np.array(users_infos_mmW)
                
    #Initialize the mean achieved rate & the probability arrays for the reward
    achieved_means = np.zeros(K_total)
    successfulRate_epoch = np.zeros(K_total)
    failDelay_epoch = np.zeros(K_total)
    average_delay = np.zeros(K_total)
    delay_means = np.zeros(K_total)
    
    
    #print(SINR_sub6)
    #print(SINR_mmW)
    ##Start Opti

    
    total_achieved = np.zeros((max_epoch, K_total))
    total_delay = np.zeros((max_epoch, K_total))

    for i in range(max_epoch):

        previousFval_sub6 = 0
        previousFval_mmW = 0
        
        ## Points initials de l'optim (schedule initial), MODIFIER (voir matlab, fonction init_schedule2)
        # schedule_init_sub6 = np.zeros(size_sub6)
        # schedule_init_mmW = np.zeros(size_mmW)
        schedule_init_sub6 = init_schedule2(N_sub6, K_sub6, schedule_frameTime)
        schedule_init_mmW =  init_schedule2(N_mmW, K_mmW, schedule_frameTime)
        # print("schedule_init_sub6 = ",schedule_init_sub6)
        # print("schedule_init_sub6 = ",schedule_init_mmW)
        
        ## Points initials de l'optim (schedule initial), MODIFIER (voir matlab, fonction init_schedule2)
        # schedule_0_sub6 = np.zeros(size_sub6)
        # schedule_0_mmW = np.zeros(size_mmW)
     
        ## Récupérer les valeurs des canaux pour la trame i
        SINR_sub6_epoch = np.zeros(K_sub6)
        if(K_sub6 != 0):
            SINR_sub6_epoch = SINR_sub6[:, i]
    
        SINR_mmW_epoch = np.zeros(K_mmW)
        if(K_mmW != 0):
            SINR_mmW_epoch = SINR_mmW[:, i]
    
        cons_sub6 = []
        cons_mmW = []
        
        ## Vérifier contraintes
        cons_sub6 = AllCons(schedule_init_sub6, SINR_sub6_epoch, Q_inv_sub6, l, K_sub6, N_sub6, users_infos_sub6, band_per_RB)
        cons_mmW = AllCons(schedule_init_mmW, SINR_mmW_epoch, Q_inv_mmW, l, K_mmW, N_mmW, users_infos_mmW, bandwidth_mmW)
        
        
        j = 0
        ## Optim à vérifier (Algo. 2, voir matlab et article)
        while j < J_max:
            
            ## SUB 6
            if K_sub6 != 0:
                sol = minimize(U_bar, schedule_init_sub6, args=(schedule_init_sub6, SINR_sub6_epoch, Q_inv_sub6, l, K_sub6), method = 'SLSQP', constraints = cons_sub6, options={'eps':1})

                
                # print("sol.x :",sol.x)
                ## AJOUTER fonction d'arrêt si minimum local atteint (voir code matlab)
                print("sol:",transform_array(sol.x, K_sub6, N_sub6))
                if sol.fun-previousFval_sub6<optimal_tolerance:
                    break
                previousFval_sub6 = sol.fun
                scheduleOpt_sub6 = sol.x
                
                for k in range(K_sub6 * N_sub6 * schedule_frameTime):
                    schedule_init_sub6[k] = scheduleOpt_sub6[k]
    
                cons_sub6 = AllCons(schedule_init_sub6, SINR_sub6_epoch, Q_inv_sub6, l, K_sub6, N_sub6, users_infos_sub6, band_per_RB)
            
            ## mmW
            if K_mmW != 0:
                sol = minimize(U_bar, schedule_init_mmW, args=(schedule_init_mmW, SINR_mmW_epoch, Q_inv_mmW, l, K_mmW), method = 'SLSQP', constraints = cons_mmW, options={'eps':1})
            
                
                ## AJOUTER fonction d'arrêt si minimum local atteint (voir code matlab)
                if sol.fun-previousFval_mmW<optimal_tolerance:
                    break
                previousFval_mmW = sol.fun
                
                scheduleOpt_mmW = sol.x

                for k in range(K_mmW * N_mmW * schedule_frameTime):
                    schedule_init_mmW[k] = scheduleOpt_mmW[k]
    
                cons_mmW = AllCons(schedule_init_mmW, SINR_mmW_epoch, Q_inv_mmW, l, K_mmW, N_mmW, users_infos_mmW, bandwidth_mmW)

            j += 1
        print("scheduleOpt_sub6 : ", schedule_init_sub6)
        print("schedule_init_mmW : ", schedule_init_mmW)
    
        
        


        current_achieved = np.zeros(K_total)
        current_delay = np.zeros(K_total)
        
        #Compute the achieved rate for Sub-6 & mmWave:
        ## A VERIFIER, 
        ## Il faut aussi modifier cette fonction, vérifier que c'est un cas faisable (voir Matlab), si on a par ex. plusieurs utilisateurs assigné à la même ressource on ne prend pas en compte le résultat
        print("K_sub6",K_sub6)
        if(K_sub6 != 0):
            result_sub6 = schedule_init_sub6
            result_sub6[result_sub6 < 0.5] = 0
            result_sub6[result_sub6 > 0.5] = 1
            result_sub6 = transform_array(result_sub6, K_sub6, N_sub6)
            # print("\n\n\nresult_sub6 : ",result_sub6)
            # print("SINR_sub6_epoch : ",SINR_sub6_epoch)
            userperblock_sub6 = (np.sum(result_sub6, axis=2) - 1) <= 0
            # maxblock_sub6 = (np.sum(result_sub6, axis = 0) - N_sub6) <= 0
            maxblock_sub6 = (np.sum(result_sub6, axis = (0,2)) - N_sub6) <= 0
            # print("result_sub6 : ",result_sub6)
            # print("N_sub6 : ",N_sub6)
            # print("(np.sum(result_sub6, axis = 0) - N_sub6) :",(np.sum(result_sub6, axis = 0) - N_sub6))
            # print("np.sum(result_sub6, axis = 0) : ",np.sum(result_sub6, axis = 0))
            # print("condition : ",np.all(userperblock_sub6),np.all(maxblock_sub6))

            result_sub6 = result_sub6.flatten()
            if np.all(userperblock_sub6) and np.all(maxblock_sub6):

                # optimal_val_sub6[i] = fval_sub6

                # Compute the achieved rate for this epoch's solution
                cap_sub6 = F_k(result_sub6, SINR_sub6_epoch, band_per_RB, schedule_frameTime, N_sub6, K_sub6)
                disp_sub6 = V_k(result_sub6, SINR_sub6_epoch, Q_inv_sub6, l, band_per_RB, schedule_frameTime, N_sub6, K_sub6)

                # Compute the achieved delay for this epoch's solution
                print(result_sub6, K_sub6, N_sub6, schedule_frameTime)
                current_delay_sub6 = delay(result_sub6, K_sub6, N_sub6, schedule_frameTime)
                print("current_delay_sub6",current_delay_sub6)
                # break

            else:
                cap_sub6 = np.zeros(K_sub6)
                disp_sub6 = np.zeros(K_sub6)
                current_delay_sub6 = np.zeros(K_sub6)
                for i in range(K_sub6):
                    current_delay_sub6[i] = schedule_frameTime + 1
            # print("cap_sub6 : ",cap_sub6)
            # print("disp_sub6 : ",disp_sub6)
            # print("cap_sub6-disp_sub6 : ",cap_sub6-disp_sub6)
            print("current_delay_sub6 : ",np.all(userperblock_sub6) and np.all(maxblock_sub6),current_delay_sub6)
        
        ## Pareil qu'au dessus, vérifier et modifier
        if(K_mmW != 0):
            result_mmW = schedule_init_mmW
            result_mmW[result_mmW < 0.5] = 0
            result_mmW[result_mmW > 0.5] = 1
            result_mmW = transform_array(result_mmW, K_mmW, N_mmW)
            userperblock_mmW = (np.sum(result_mmW, axis=2) - 1) <= 0
            maxblock_mmW = (np.sum(result_mmW, axis = (0,2)) - N_mmW) <= 0

            result_mmW = result_mmW.flatten()
            if np.all(userperblock_mmW) and np.all(maxblock_mmW):

                # Compute the achieved rate for this epoch's solution
                cap_mmW = F_k(result_mmW, SINR_mmW_epoch, band_per_RB, schedule_frameTime, N_mmW, K_mmW)
                disp_mmW = V_k(result_mmW, SINR_mmW_epoch, Q_inv_mmW, l, band_per_RB, schedule_frameTime, N_mmW, K_mmW)

                # Compute the achieved delay for this epoch's solution
                current_delay_mmW = delay(result_mmW, K_mmW, N_mmW, schedule_frameTime)

            else:
                cap_mmW = np.zeros(K_mmW)
                disp_mmW = np.zeros(K_mmW)
                current_delay_mmW = np.zeros(K_mmW)
                for i in range(K_mmW):
                    current_delay_mmW[i] = schedule_frameTime + 1
            
        cpt_s6 = 0
        cpt_mmW = 0
        print("K_total",K_total)
        for k in range(K_total):
            if sub6_users[k] == 1:
                current_achieved[k] = cap_sub6[cpt_s6] - disp_sub6[cpt_s6]
                current_delay[k] = current_delay_sub6[cpt_s6]
                cpt_s6 += 1
            if mmW_users[k] == 1:
                current_achieved[k] = cap_mmW[cpt_mmW] - disp_mmW[cpt_mmW]
                current_delay[k] = current_delay_mmW[cpt_mmW]
                cpt_mmW += 1
        print(current_delay)
                

        ## Vérifier que c'est correct, voir code Matlab

        
        current_delay[current_delay == 0] = schedule_frameTime + 1
        print(current_delay)
        delay_means = delay_means + (current_delay - delay_means)/(i+1)

        achieved_means = achieved_means + (current_achieved - achieved_means)/(i+1)
        successfulRate_epoch = successfulRate_epoch + (achieved_means < users_infos[1,:])
        print("failDelay_epoch")
        print(failDelay_epoch, current_delay, users_infos[3,:])
        failDelay_epoch = failDelay_epoch + (current_delay > users_infos[3,:])
        
        total_achieved[i,:] = current_achieved
        total_delay[i,:] = current_delay
        
        #Update the user requirements
        cpt_sub6 = 0
        cpt_mmW = 0
        for x in range(K_total) : 
            if(sub6_users[x] == 1):
                users_infos_sub6[cpt_sub6,1] = (i+2) * max(0, users_infos[1,x] - achieved_means[x]) + users_infos[1,x]
                cpt_sub6 += 1
            if(mmW_users[x] == 1):
                users_infos_mmW[cpt_mmW,1] = (i+2) * max(0, users_infos[1,x] - achieved_means[x]) + users_infos[1,x]
                cpt_mmW += 1
        print("users_infos_sub6", users_infos_sub6)
        print("users_infos_mmW",users_infos_mmW)
    
    achieved = achieved_means
    prob_rates = successfulRate_epoch / max_epoch
    prob_delay = failDelay_epoch / max_epoch
    average_delay = delay_means
    time_opti = time.time() - start_time
    
    print("achieved dans opti: ", achieved)
    print("prob rates : ", prob_rates)
    print("prob delay : ", prob_delay)
    
    return achieved, prob_rates, prob_delay, average_delay, time_opti, total_achieved, total_delay
# print(delay([1,1,0,0,1,1,0,0],2,2,2))
scheduler([1,1,1,1],1,1)

sol: [[[ 1.24364728e-03  1.24322679e-03  1.24364754e-03  1.00372954e+00]
  [ 1.24319990e-03  1.24311338e-03  1.00372955e+00  1.24311337e-03]
  [ 1.24322173e-03  1.24320712e-03  1.24355451e-03  1.00373000e+00]
  [ 1.24355450e-03  1.24362908e-03  1.72899693e-07  1.00373001e+00]
  [ 1.72913790e-07  1.00373000e+00  1.72906060e-07  1.72912290e-07]]

 [[ 1.24377461e-03  1.24377396e-03  1.24374842e-03  1.00373022e+00]
  [ 1.00124627e+00  1.00373022e+00  1.24389213e-03  1.24378372e-03]
  [ 1.00373022e+00  1.00124627e+00  1.24323660e-03  1.24348868e-03]
  [ 1.24323897e-03  1.00372966e+00 -1.72920800e-07  1.24321060e-03]
  [-1.72928075e-07  1.00372966e+00 -1.72919436e-07 -1.72863523e-07]]]
scheduleOpt_sub6 :  [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
schedule_init_mmW :  []
K_sub6 4
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.] 4 2 5
curren

(array([87953987.31967583, 45702641.11557274, 27695392.10164425,
        20728530.67124156]),
 array([1. , 0.5, 1. , 0.2]),
 array([0. , 0. , 0.3, 0. ]),
 array([2.5, 4. , 2.8, 2.4]),
 1.558499813079834,
 array([[ 2.27746921e+05,  6.79602279e+07,  2.57100136e+07,
          9.83791422e+06],
        [ 8.91112948e+07,  3.12242098e+07,  8.11653139e+06,
          1.18134339e+07],
        [ 5.96778108e+07,  2.99496640e+07,  0.00000000e+00,
          9.94861326e+06],
        [ 5.64667657e+07,  1.15817312e+08,  3.22880107e+07,
          8.10404884e+06],
        [ 9.57084415e+07,  1.41859701e+07,  4.96727373e+06,
          6.20122704e+07],
        [ 2.18414847e+08,  4.59649520e+07,  4.74104074e+07,
          1.94029720e+06],
        [ 1.05623441e+08,  6.42815074e+06,  1.65968973e+07,
          3.28894527e+07],
        [ 8.35395426e+07,  1.43961557e+07,  4.92153070e+07,
         -2.63025622e+05],
        [ 1.08887697e+08,  1.26108815e+08,  2.85291994e+07,
          7.41714212e+06],
        [ 6.1